# [Can You Play the Favorite?](https://thefiddler.substack.com/p/can-you-play-the-favorite)
## March 21 2025

## Problem

_Dean has three colors of the hibiscus: red, orange, and yellow. He wants to plant them in a straight hedge of shrubs (each of which is one color) so that the order appears somewhat random, but not truly random. More specifically, he wants the following to be true:_

_No two adjacent shrubs have the same color._

_No ordering of three consecutive shrubs appears more than once in the hedge. (But a prior ordering can appear in reverse. For example, ROYOR is an acceptable hedge, but ROYROY is not.)_

_What is the greatest number of shrubs Dean’s hedge can contain?_

## Solution

We can model the problem as a graph traversal problem, much like last week's puzzle. The most obvious (in my opinion) way to do this is to represent each possible ordered triple (of which there are $3 \times 2 \times 2 = 12$) as a node, and draw a directed edge from node 1 to node 2 if the last two elements of node 1 equal the first two elements of node 2. Then we check if there's a Hamiltonian path (or, if there isn't, find the longest path admitted by the graph). This could work, but as best I can tell, there aren't any nice tidy theorems that tell us whether this graph has a Hamiltonian path (something like Dirac's theorem of Hamiltonicity). Instead, we would have to find a Hamiltonian path, probably with the help of a computer and a path-finding algorithm.

However, if we format the graph a bit differently, we can easily show that there is a way to include all possible ordered triples in the shrub sequence. If we instead generate all possible ordered pairs of three elements and connect node 1 to node 2 if node 1's last element is equal to node 2's first element, then we have a graph where each edge represents the possible ordered triples. If we can find a path that spans all edges without reusing any, we can use all possible ordered triples, and our answer will be easy to compute. In other words, instead of searching for a Hamiltonian path, we're searching for an Eulerian path. We know that if every node of a directed graph has in-degree equal to out-degree, then there exists an Eulerian cycle, which implies an Eulerian path.

Hopefully it's pretty clear that this pair-type graph has in-degree equal to out-degree for every vertex. There are a total of 6 pairs for a given vertex. A vertex will point to exactly two other nodes (out-degree), since the last element of the source vertex must equal the first element of the target vertex. By symmetry, the same is true for the in-degree.

Also, we can extend this reasoning for the cases when we're considering ordered sets of any size (not just 3). So if our requirement is that no $N$ consecutive shrubs appear more than once in a hedge, we can always construct a sequence that includes all possible ordered pairs of length $N$.

All that's left is to calculate the actual number of shrubs. This is easy. We start with $N$ shrubs for the first set of consecutive shrubs. Then we add 1 for each of the remaining distinct ordered sets. For an ordered set of size $N$ with replacement, it has size $N \times (N - 1)^{N - 1}$. So our total is $N + N \times (N - 1)^{N - 1} - 1$. For $N = 3$ that's $\boxed{14}$, and for $N = 4$ that's $\boxed{111}$.

The code below can prove computationally that such a path exists.

In [20]:
import networkx as nx
from itertools import product

def create_nodes_for_orderings(elements, n):
    """
    Creates a graph with nodes representing valid n-element orderings where
    adjacent elements cannot be the same. Each node stores the ordering.
    
    Args:
        elements: Iterable of elements to create orderings from
        n: Length of each ordering
        
    Returns:
        NetworkX graph with nodes representing valid orderings
    """
    G = nx.Graph()
    
    # Generate all possible combinations with replacement
    for ordering in product(elements, repeat=n):
        # Check if any adjacent elements are the same
        is_valid = True
        for i in range(n-1):
            if ordering[i] == ordering[i+1]:
                is_valid = False
                break
                
        if is_valid:
            # Convert ordering to tuple to ensure it's hashable
            ordering_key = tuple(ordering)
            # Add node with the ordering stored as a node attribute
            G.add_node(ordering_key, ordering=ordering_key)
    
    return G

def create_directed_edges(G):
    """
    Creates directed edges between nodes where the last n-1 elements of the source node
    match the first n-1 elements of the target node.
    
    Args:
        G: NetworkX graph containing nodes with orderings
        
    Returns:
        NetworkX DiGraph with the directed edges added
    """
    # Convert to directed graph if it isn't already
    if not isinstance(G, nx.DiGraph):
        DG = nx.DiGraph(G)
    else:
        DG = G
        
    # Get length of orderings from first node
    n = len(next(iter(DG.nodes())))
    
    # For each pair of nodes
    for source in DG.nodes:
        for target in DG.nodes:
            # Compare last n-1 elements of source with first n-1 elements of target
            if source[1:] == target[:-1]:
                DG.add_edge(source, target)
                
    return DG

def find_hamiltonian_path(G, start_node=None):
    """
    Finds a Hamiltonian path in a directed graph using optimized backtracking.
    
    Args:
        G: NetworkX DiGraph
        start_node: Optional starting node. If None, tries each node until a path is found
        
    Returns:
        List representing the Hamiltonian path if one exists, None otherwise
    """
    def try_path(current, path, visited):
        # If we've visited all nodes, we've found a Hamiltonian path
        if len(path) == len(G):
            return path
        
        # Get all possible next nodes, sorted by out-degree (optimization)
        neighbors = sorted(
            [n for n in G.successors(current) if n not in visited],
            key=lambda x: G.out_degree(x),  # Prefer nodes with fewer options first
            reverse=True
        )
        
        # Try each unvisited neighbor
        for next_node in neighbors:
            visited.add(next_node)
            path.append(next_node)
            
            result = try_path(next_node, path, visited)
            if result is not None:
                return result
                
            # Backtrack
            visited.remove(next_node)
            path.pop()
            
        return None

    # If no start node specified, try nodes in order of out-degree
    if start_node is None:
        # Sort nodes by out-degree to try most promising starts first
        start_nodes = sorted(
            G.nodes(), 
            key=lambda x: G.out_degree(x),
            reverse=True
        )
    else:
        start_nodes = [start_node]

    # Try each possible starting node
    for start in start_nodes:
        path = [start]
        visited = {start}
        result = try_path(start, path, visited)
        if result is not None:
            return result

    return None

# Example usage:
def verify_hamiltonian_path(G, path):
    """
    Verifies that a path is indeed Hamiltonian in the given graph.
    """
    if path is None:
        return False
        
    # Check if all nodes are included exactly once
    if len(set(path)) != len(G) or len(path) != len(G):
        return False
        
    # Check if each consecutive pair forms an edge in the graph
    for i in range(len(path)-1):
        if not G.has_edge(path[i], path[i+1]):
            return False
            
    return True


# Example usage:
n = 4
elements = range(1, n+1)
G = create_nodes_for_orderings(elements, n)
DG = create_directed_edges(G)

# Verify nodes
print(f"Number of nodes: {len(DG.nodes)}")

# Verify edges
print(f"Number of edges: {len(DG.edges)}")

# # Print some example edges
# print("\nSample of edges (first 5):")
# for i, (source, target) in enumerate(list(DG.edges)[:9]):
#     print(f"Edge {i+1}: {source} -> {target}")

# Get a sample of nodes (first 5 nodes)
sample_nodes = list(DG.nodes())[:5]

print("Degree Analysis:")
print("-" * 50)
print(f"{'Node':<20} {'In-Deg':>8} {'Out-Deg':>8} {'Total':>8}")
print("-" * 50)

for node in sample_nodes:
    in_deg = DG.in_degree(node)
    out_deg = DG.out_degree(node)
    total_deg = in_deg + out_deg
    print(f"{str(node):<20} {in_deg:>8} {out_deg:>8} {total_deg:>8}")

# Calculate averages for the whole graph
avg_in = sum(dict(DG.in_degree()).values()) / DG.number_of_nodes()
avg_out = sum(dict(DG.out_degree()).values()) / DG.number_of_nodes()
avg_total = avg_in + avg_out

print("\nGraph Averages:")
print(f"Average in-degree:  {avg_in:.2f}")
print(f"Average out-degree: {avg_out:.2f}")
print(f"Average total:      {avg_total:.2f}")

# Example usage:
path = find_hamiltonian_path(DG)
if path:
    print("Found Hamiltonian path:", path)
    print("Path is valid:", verify_hamiltonian_path(DG, path))
else:
    print("No Hamiltonian path exists")

# Optional: Print the first few transitions in the path
if path:
    print("\nFirst few transitions:")
    for i in range(min(5, len(path)-1)):
        print(f"{path[i]} -> {path[i+1]}")


Number of nodes: 108
Number of edges: 324
Degree Analysis:
--------------------------------------------------
Node                   In-Deg  Out-Deg    Total
--------------------------------------------------
(1, 2, 1, 2)                3        3        6
(1, 2, 1, 3)                3        3        6
(1, 2, 1, 4)                3        3        6
(1, 2, 3, 1)                3        3        6
(1, 2, 3, 2)                3        3        6

Graph Averages:
Average in-degree:  3.00
Average out-degree: 3.00
Average total:      6.00
Found Hamiltonian path: [(1, 2, 1, 2), (2, 1, 2, 1), (1, 2, 1, 3), (2, 1, 3, 1), (1, 3, 1, 2), (3, 1, 2, 1), (1, 2, 1, 4), (2, 1, 4, 1), (1, 4, 1, 2), (4, 1, 2, 3), (1, 2, 3, 1), (2, 3, 1, 2), (3, 1, 2, 3), (1, 2, 3, 2), (2, 3, 2, 1), (3, 2, 1, 2), (2, 1, 2, 3), (1, 2, 3, 4), (2, 3, 4, 1), (3, 4, 1, 2), (4, 1, 2, 4), (1, 2, 4, 1), (2, 4, 1, 3), (4, 1, 3, 1), (1, 3, 1, 3), (3, 1, 3, 1), (1, 3, 1, 4), (3, 1, 4, 1), (1, 4, 1, 3), (4, 1, 3, 2), (1, 3, 2, 1), 